In [1]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, contact_optimization, get_double_side_view, show_selected_joints, highlight_rod_and_joint)
import vis.fields
import matplotlib.cm as cm
import time

In [2]:
import parallelism
parallelism.set_max_num_tbb_threads(24)
parallelism.set_hessian_assembly_num_threads(8)
parallelism.set_gradient_assembly_num_threads(8)

In [3]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.01
dsw = 0.01

In [4]:
import pickle
import gzip

In [5]:
rod_length = 0.09982061776030171
width = rod_length / 15 * 5
thickness = width / 5 * 0.35
print(width, thickness)
io = InputOrganizer('kitty_level_0', thickness, width, weaving_dir)
#  [ ear 214, ear 255, nose 39]

0.033273539253433904 0.0023291477477403733


In [6]:
# import pickle
# import gzip

In [7]:
# pickle.dump(curved_linkage, gzip.open('{}_stage_2_20201230.pkl.gz'.format(io.MODEL_NAME), 'w'))

In [ ]:
list_of_cat = ['new_kitty_1', 'kitty_level_0', 'kitty_level_1', 'kitty_level_4', 'kitty_level_99']

In [ ]:
list_of_linkage = []
list_of_length = []
list_of_curvature = []
list_of_all_curvature = []
for cat_name in list_of_cat:
    curved_linkage = pickle.load(gzip.open('{}_stage_2_20201230.pkl.gz'.format(cat_name), 'r'))
    list_of_linkage.append(curved_linkage)
    ribbons = order_segments_by_ribbons(curved_linkage)
    all_ribbon_angle, all_ribbon_edge_len, all_ribbon_num_seg, all_ribbon_widths, all_joint_index, _ = get_turning_angle_and_length_from_ordered_rods(ribbons, curved_linkage, rest = True, bending_axis = 0)
    segment_indices = [x[0] for x in ribbons[5]]
    info = curved_linkage.getPerSegmentRestLength()[segment_indices]
    list_of_length.append(info)
    list_of_curvature.append(all_ribbon_angle[5])
    list_of_all_curvature.append(all_ribbon_angle)

In [ ]:
import seaborn as sns

In [ ]:
list_of_curvature = np.array(list_of_curvature)

In [ ]:
labels = ['cat', 'transition 1', 'transition 2', 'transition 3', 'sphere']

### Bad visualization

In [ ]:


# Make the histogram using a list of lists
# Normalize the flights and assign colors and names
plt.hist(np.transpose(list_of_curvature), bins = int(180/15), normed=True, label=labels)

# Plot formatting
plt.legend()

In [ ]:
plt.hist(np.transpose(list_of_curvature), bins = int(180/15), normed=True,
         stacked=True,label=labels)


In [ ]:

for i, airline in enumerate(list_of_curvature):
    # Subset to the airline
    # Make a histogram with the airline
    sns.distplot(airline, bins = int(180/5), 
                 label = labels[i], hist = True, kde = False, 
                 norm_hist=True, 
                 hist_kws = {'edgecolor': 'black'})
    
plt.legend(prop={'size': 16})

### Good visualization

In [ ]:
import pandas as pd

In [ ]:
fig, host = plt.subplots()
data = np.transpose(list_of_length)
data = pd.DataFrame(data, columns=np.array(labels))
sns.kdeplot(data = data)
plt.title('Density Plot of Segment Length of Ribbon 5')
plt.legend(prop={'size': 12}, title = 'Models')
plt.xlabel('Segment Length', fontsize=14); 
plt.ylabel('Density', fontsize=14);
plt.savefig('./cat_segment_length_density.svg', format='svg', dpi = 200)

In [ ]:
fig, host = plt.subplots()
data = np.transpose(list_of_curvature)
data = pd.DataFrame(data, columns=np.array(labels))
sns.kdeplot(data = data)
plt.title('Density Plot of Curvature of Ribbon 5')
# plt.legend(prop={'size': 12}, title = 'Models')
plt.xlabel('Discrete Curvature', fontsize=14); 
plt.ylabel('Density', fontsize=14);
plt.savefig('./cat_segment_curvature_density.svg', format='svg', dpi = 200)